In [18]:
import faiss
import torch
import ipywidgets as widgets
from IPython.display import display, Markdown
import numpy as np
import os
from multiprocessing import cpu_count
import threading
from subprocess import Popen, PIPE
import sys
from time import sleep
from config_moj import Config
from random import shuffle
from my_utils import load_audio, CSVutil
import traceback, pdb
from sklearn.cluster import MiniBatchKMeans
import math
import signal
import threading
import re

config = Config()

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

now_dir = os.getcwd()
sys.path.append(now_dir)

sr_dict = {
    "32k": 32000,
    "40k": 40000,
    "48k": 48000,
}

def if_done(done, p):
    while 1:
        if p.poll() is None:
            sleep(0.5)
        else:
            break
    done[0] = True


def if_done_multi(done, ps):
    while 1:
        flag = 1
        for p in ps:
            if p.poll() is None:
                flag = 0
                sleep(0.5)
                break
        if flag == 1:
            break
    done[0] = True

isinterrupted = 0

if torch.cuda.is_available() or ngpu != 0:
    for i in range(ngpu):
        gpu_name = torch.cuda.get_device_name(i)
        if any(
            value in gpu_name.upper()
            for value in [
                "10",
                "16",
                "20",
                "30",
                "40",
                "A2",
                "A3",
                "A4",
                "P4",
                "A50",
                "500",
                "A60",
                "70",
                "80",
                "90",
                "M4",
                "T4",
                "TITAN",
            ]
        ):
            # A10#A100#V100#A40#P40#M40#K80#A4500
            if_gpu_ok = True
            gpu_infos.append("%s\t%s" % (i, gpu_name))
            mem.append(
                int(
                    torch.cuda.get_device_properties(i).total_memory
                    / 1024
                    / 1024
                    / 1024
                    + 0.4
                )
            )
if if_gpu_ok and len(gpu_infos) > 0:
    gpu_info = "\n".join(gpu_infos)
    default_batch_size = min(mem) // 2
else:
    gpu_info = "Nie masz wspieranej karty graficznej do obsługi treningu."
    default_batch_size = 1
gpus = "-".join([i[0] for i in gpu_infos])

global log_interval


def set_log_interval(exp_dir, batch_size12):
    log_interval = 1

    folder_path = os.path.join(exp_dir, "1_16k_wavs")

    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        wav_files = [f for f in os.listdir(folder_path) if f.endswith(".wav")]
        if wav_files:
            sample_size = len(wav_files)
            log_interval = math.ceil(sample_size / batch_size12)
            if log_interval > 1:
                log_interval += 1

    return log_interval

def append_to_log(log, text, append=False):
    if text != "":
        if append:
            log.value += text + "\n"
        else:
            log.value = text

def preprocess_dataset(button):
    trainset_dir = trainset_dir_widget.value
    exp_dir = exp_dir_widget.value
    sr = sr_widget.value
    n_p = n_p_widget.value
    output_info_widget1.value = ""

    sr = sr_dict[sr]
    os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
    f = open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "w")
    f.close()
    cmd = (
        config.python_cmd + " trainset_preprocess_pipeline_print.py %s %s %s %s/logs/%s "
        % (trainset_dir, sr, n_p, now_dir, exp_dir)
        + str(config.noparallel)
    )
    print(cmd)
    p = Popen(cmd, shell=True)
    done = [False]
    threading.Thread(
        target=if_done,
        args=(
            done,
            p,
        ),
    ).start()
    while 1:
        with open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "r") as f:
            append_to_log(output_info_widget1, f.read())
        sleep(1)
        if done[0]:
            break
    with open("%s/logs/%s/preprocess.log" % (now_dir, exp_dir), "r") as f:
        log = f.read()
    print(log)
    append_to_log(output_info_widget1, log)

def extract_f0_feature(button):
    gpus = gpu_num_widget.value
    n_p = n_p_widget.value
    f0method = f0_method_widget.value
    if_f0 = pitch_guidance_widget.value
    exp_dir = exp_dir_widget.value
    version19 = rvc_ver_widget.value
    echl = crepe_hop_widget.value
    output_info_widget2.value = ""

    gpus = gpus.split("-")
    os.makedirs("%s/logs/%s" % (now_dir, exp_dir), exist_ok=True)
    f = open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "w")
    f.close()
    if if_f0:
        cmd = config.python_cmd + " extract_f0_print.py %s/logs/%s %s %s %s" % (
            now_dir,
            exp_dir,
            n_p,
            f0method,
            echl,
        )
        print(cmd)
        p = Popen(cmd, shell=True, cwd=now_dir)
        done = [False]
        threading.Thread(
            target=if_done,
            args=(
                done,
                p,
            ),
        ).start()
        while 1:
            with open(
                "%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "r"
            ) as f:
                append_to_log(output_info_widget2, f.read())
            sleep(1)
            if done[0]:
                break
        with open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "r") as f:
            log = f.read()
        print(log)
        append_to_log(output_info_widget2, log)

    leng = len(gpus)
    ps = []
    for idx, n_g in enumerate(gpus):
        cmd = (
            config.python_cmd
            + "extract_feature_print.py %s %s %s %s %s/logs/%s %s"
            % (
                config.device, # wrzucam na sztywno, później to poprawię
                leng,
                idx,
                n_g,
                now_dir,
                exp_dir,
                version19,
            )
        )
        print(cmd)
        p = Popen(
            cmd, shell=True, cwd=now_dir
        )
        ps.append(p)
    done = [False]
    threading.Thread(
        target=if_done_multi,
        args=(
            done,
            ps,
        ),
    ).start()
    while 1:
        with open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "r") as f:
            append_to_log(output_info_widget2, f.read())
        sleep(1)
        if done[0]:
            break
    with open("%s/logs/%s/extract_f0_feature.log" % (now_dir, exp_dir), "r") as f:
        log = f.read()
    print(log)
    append_to_log(output_info_widget2, log)

def change_sr2(change):
    version = rvc_ver_widget
    sr = sr_widget
    path_str = "" if version.value == "v1" else "_v2"
    f0_str = "f0" if pitch_guidance_widget.value else ""
    pretrained_generator_path = "pretrained%s/%sG%s.pth" % (path_str, f0_str, sr.value)
    pretrained_discriminator_path = "pretrained%s/%sD%s.pth" % (path_str, f0_str, sr.value)
    if_pretrained_generator_exist = os.access(
        pretrained_generator_path, os.F_OK
    )
    if_pretrained_discriminator_exist = os.access(
        pretrained_discriminator_path, os.F_OK
    )
    if not if_pretrained_generator_exist:
        print(
            pretrained_generator_path,
            "doesn't exist, will not use pretrained model",
        )
    if not if_pretrained_discriminator_exist:
        print(
            pretrained_discriminator_path,
            "doesn't exist, will not use pretrained model",
        )
    pretrained_g_model.value = pretrained_generator_path \
        if if_pretrained_generator_exist \
        else ""
    pretrained_d_model.value = pretrained_discriminator_path \
        if if_pretrained_discriminator_exist \
        else ""


def change_version19(change):
    version = rvc_ver_widget
    sr = sr_widget
    path_str = "" if version.value == "v1" else "_v2"
    tmp_sr = sr.value
    if sr.value == "32k" and version.value  == "v1":
        tmp_sr = "40k"
    sr.options = ["40k", "48k"] if version.value == "v1" else ["40k", "48k", "32k"]
    sr.value = tmp_sr
    f0_str = "f0" if pitch_guidance_widget.value else ""
    pretrained_generator_path = "pretrained%s/%sG%s.pth" % (path_str, f0_str, sr.value)
    pretrained_discriminator_path = "pretrained%s/%sD%s.pth" % (path_str, f0_str, sr.value)
    if_pretrained_generator_exist = os.access(
        pretrained_generator_path, os.F_OK
    )
    if_pretrained_discriminator_exist = os.access(
        pretrained_discriminator_path, os.F_OK
    )
    if not if_pretrained_generator_exist:
        print(
            pretrained_generator_path,
            "doesn't exist, will not use pretrained model",
        )
    if not if_pretrained_discriminator_exist:
        print(
            pretrained_discriminator_path,
            "doesn't exist, will not use pretrained model",
        )

    pretrained_g_model.value = pretrained_generator_path \
        if if_pretrained_generator_exist \
        else ""
    pretrained_d_model.value = pretrained_discriminator_path \
        if if_pretrained_discriminator_exist \
        else ""

def change_f0(change):
    version = rvc_ver_widget
    sr = sr_widget
    path_str = "" if version.value == "v1" else "_v2"
    f0_str = "f0" if pitch_guidance_widget.value else ""
    pretrained_generator_path = "pretrained%s/%sG%s.pth" % (path_str, f0_str, sr.value)
    pretrained_discriminator_path = "pretrained%s/%sD%s.pth" % (path_str, f0_str, sr.value)
    if_pretrained_generator_exist = os.access(
        pretrained_generator_path, os.F_OK
    )
    if_pretrained_discriminator_exist = os.access(
        pretrained_discriminator_path, os.F_OK
    )
    if not if_pretrained_generator_exist:
        print(
            pretrained_generator_path,
            "not exist, will not use pretrained model",
        )
    if not if_pretrained_discriminator_exist:
        print(
            pretrained_discriminator_path,
            "not exist, will not use pretrained model",
        )

    pretrained_g_model.value = pretrained_generator_path if if_pretrained_generator_exist else ""
    pretrained_d_model.value = pretrained_discriminator_path if if_pretrained_discriminator_exist else ""

    if pitch_guidance_widget.value:
        f0_method_widget.layout.display = "block"
        tab.children[0].children[3].children[0].layout.display = "block"
        gpu_num_widget.layout.display = "block"
        gpu_info_widget.layout.display = "block"
        crepe_hop_widget.layout.display = "block"
        f0_process_btn.layout.display = "block"
        output_info_widget2.layout.display = "block"
    else:
        f0_method_widget.layout.display = "none"
        tab.children[0].children[3].children[0].layout.display = "none"
        gpu_num_widget.layout.display = "none"
        gpu_info_widget.layout.display = "none"
        crepe_hop_widget.layout.display = "none"
        f0_process_btn.layout.display = "none"
        output_info_widget2.layout.display = "none"

def stoptraining(mim):
    #CSVutil("csvdb/stop.csv", "w+", "stop", "True")
    # p.terminate()
    # p.kill()
    
    switch_train_button()

    try:
        os.kill(PID, signal.SIGTERM)
    except Exception as e:
        print(f"Couldn't click due to {e}")
        pass



def whethercrepeornah(radio):
    mango = "visible" if radio == "mangio-crepe" or radio == "mangio-crepe-tiny" else "hidden"

    crepe_hop_widget.layout.visibility = mango

def switch_train_button():
    if train_model_btn.description == "Trenuj model":
        train_model_btn.description = "Zatrzymaj trenowanie"
        train_model_btn.unobserve_all()
        train_model_btn.on_click(stoptraining)
    else:
        train_model_btn.description = "Trenuj model"
        train_model_btn.unobserve_all()
        train_model_btn.on_click(click_train)

def reader_thread(pipe, process, widget):
    while True:
        line = pipe.readline()
        if line:
            line = line.strip()
            print(line)
            append_to_log(widget, line, True)
            match = re.search(r'Epoch: (\d+)', line)
            if match:
                current_epoch = int(match.group(1))
                training_progress_bar.value = current_epoch
                training_progress_bar.description = "Epoch: %d/%d" % (current_epoch, training_progress_bar.max)
        elif process.poll() is not None:
            break
    pipe.close()

def click_train(button):
    exp_dir1 = exp_dir_widget.value
    sr2 = sr_widget.value
    if_f0_3 = pitch_guidance_widget.value
    spk_id5 = spk_id_widget.value
    save_epoch10 = save_interval_widget.value
    total_epoch11 = epochs_widget.value
    batch_size12 = batch_size_widget.value
    if_save_latest13 = save_latest_widget.value
    pretrained_G14 = pretrained_g_model.value
    pretrained_D15 = pretrained_d_model.value
    gpus16 = gpu_num_widget.value
    if_cache_gpu17 = cache_dataset_widget.value
    if_save_every_weights18 = save_small_model_widget.value
    version19 = rvc_ver_widget.value

    output_info_widget3.value = ""

    switch_train_button()
    training_progress_bar.layout.visibility = "visible"
    training_progress_bar.value = 0
    training_progress_bar.max = total_epoch11
    training_progress_bar.description = "Epoch: %d/%d" % (0, total_epoch11)

    #CSVutil("csvdb/stop.csv", "w+", "formanting", False)
    # 生成filelist
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )

    log_interval = set_log_interval(exp_dir, batch_size12)

    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))
    print("write filelist done")
    # 生成config#无需生成config
    # cmd = python_cmd + " train_nsf_sim_cache_sid_load_pretrain.py -e mi-test -sr 40k -f0 1 -bs 4 -g 0 -te 10 -se 5 -pg pretrained/f0G40k.pth -pd pretrained/f0D40k.pth -l 1 -c 0"
    print("use gpus:", gpus16)
    if pretrained_G14 == "":
        print("no pretrained Generator")
    if pretrained_D15 == "":
        print("no pretrained Discriminator")
    if gpus16:
        ####
        cmd = (
            config.python_cmd
            + " train_nsf_sim_cache_sid_load_pretrain.py -e %s -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s"
            % (
                exp_dir1,
                sr2,
                1 if if_f0_3 else 0,
                batch_size12,
                gpus16,
                total_epoch11,
                save_epoch10,
                "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
                "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
                1 if if_save_latest13 == True else 0,
                1 if if_cache_gpu17 == True else 0,
                1 if if_save_every_weights18 == True else 0,
                version19,
                log_interval,
            )
        )
    else:
        cmd = (
            config.python_cmd
            + " train_nsf_sim_cache_sid_load_pretrain.py -e %s -sr %s -f0 %s -bs %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s"
            % (
                exp_dir1,
                sr2,
                1 if if_f0_3 else 0,
                batch_size12,
                total_epoch11,
                save_epoch10,
                "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "\b",
                "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "\b",
                1 if if_save_latest13 == True else 0,
                1 if if_cache_gpu17 == True else 0,
                1 if if_save_every_weights18 == True else 0,
                version19,
                log_interval,
            )
        )
    print(cmd)
    global p
    #p = Popen(cmd, shell=True, cwd=now_dir)
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=PIPE, universal_newlines=True, bufsize=1)

    global PID
    PID = p.pid

    stdout_thread = threading.Thread(target=reader_thread, args=[p.stdout, p, output_info_widget3])
    stderr_thread = threading.Thread(target=reader_thread, args=[p.stderr, p, output_info_widget3])

    stdout_thread.start()
    stderr_thread.start()

    stdout_thread.join()
    stderr_thread.join()

    p.wait()
    append_to_log(output_info_widget3, "Po zakończeniu treningu można wyświetlić dziennik treningu na konsoli lub plik train.log w folderze eksperymentu.")
    switch_train_button()

def train_index(button):
    exp_dir1 = exp_dir_widget.value
    version19 = rvc_ver_widget.value
    output_info_widget3.value = ""

    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        append_to_log(output_info_widget3, "Najpierw wykonaj ekstrakcję cech!")
        return
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        append_to_log(output_info_widget3, "Najpierw wykonaj ekstrakcję cech!")
        return
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        # if(1):
        append_to_log(output_info_widget3, "Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0], True)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            print(info)
            append_to_log(output_info_widget3, info, True)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    append_to_log(output_info_widget3, "%s,%s" % (big_npy.shape, n_ivf), True)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    # index = faiss.index_factory(256if version19=="v1"else 768, "IVF%s,PQ128x4fs,RFlat"%n_ivf)
    append_to_log(output_info_widget3, "training", True)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    # faiss.write_index(index, '%s/trained_IVF%s_Flat_FastScan_%s.index'%(exp_dir,n_ivf,version19))
    append_to_log(output_info_widget3, "adding", True)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    append_to_log(output_info_widget3, "Successful Index Construction，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19), True)
    # faiss.write_index(index, '%s/added_IVF%s_Flat_FastScan_%s.index'%(exp_dir,n_ivf,version19))
    # infos.append("成功构建索引，added_IVF%s_Flat_FastScan_%s.index"%(n_ivf,version19))


Found GPU NVIDIA GeForce RTX 3060
Set fp16_run to true in 32k.json
Set fp16_run to true in 40k.json
Set fp16_run to true in 48k.json


In [20]:
box_layout = widgets.Layout(display='flex', flex_flow='column', align_items='stretch', border='solid', width='50%')

# przyciski
preprocess_btn = widgets.Button(
                    description="Przetwarzaj dane",
                    button_style="primary"
                )
preprocess_btn.on_click(preprocess_dataset)

f0_process_btn = widgets.Button(
    description="Ekstrakcja cech",
    button_style="primary"
)

f0_process_btn.on_click(extract_f0_feature)

train_model_btn = widgets.Button(
    description="Trenuj model",
    button_style="primary",
    visible=True
)

train_model_btn.on_click(click_train)

train_index_btn = widgets.Button(
    description="Trenuj plik indeksu",
    button_style="primary"
)

train_index_btn.on_click(train_index)

# kontrolki i observery
trainset_dir_widget = widgets.Text(
    description="Wprowadź ścieżkę folderu z datasetem",
    value=os.path.abspath(os.getcwd()) + "/datasets/",
    layout=widgets.Layout(width='300px', height='300px')
)

exp_dir_widget = widgets.Text(
    description="Wprowadź nazwę modelu",
    value="mi-test"
)

sr_widget = widgets.RadioButtons(
    description="Docelowa częstotliwość próbkowania",
    options=["40k", "48k"],
    value="40k"
)
sr_widget.observe(change_sr2, names='value')

pitch_guidance_widget = widgets.Checkbox(
    description="Czy model ma wsparcie dla wysokości dźwięku?",
    value=True
)
pitch_guidance_widget.observe(change_f0, names='value')

rvc_ver_widget = widgets.RadioButtons(
    description="Wersja",
    options=["v1", "v2"],
    value="v1",
    visible=True
)
rvc_ver_widget.observe(change_version19, names='value')

n_p_widget = widgets.IntSlider(
    description="Liczba procesów CPU używanych do ekstrakcji wysokości dźwięku i przetwarzania danych",
    min=0,
    max=cpu_count(),
    step=1,
    value=int(np.ceil(cpu_count() / 1.5))
)

spk_id_widget = widgets.IntSlider(
    description="Proszę podać id mówcy",
    min=0,
    max=4,
    step=1,
    value=0
)

gpu_num_widget = widgets.Text(
    description="Podaj numery kart oddzielone znakiem '-', na przykład 0-1-2 oznacza użycie kart 0, 1 i 2",
    value=gpus,
    interactive=True
)

f0_method_widget = widgets.RadioButtons(
    description="Wybierz algorytm ekstrakcji wysokości dźwięku: pm (szybki, dobrej jakości głos), dio (szybki, ale gorszy na słabszych CPU), harvest (lepsza jakość, ale wolniejszy)",
    options=["pm", "harvest", "dio", "crepe", "mangio-crepe", "rmvpe"],
    value="rmvpe",
    interactive=True
)
f0_method_widget.observe(whethercrepeornah, names='value')

crepe_hop_widget = widgets.IntSlider(
    description="Długość skoku Mangio-Crepe (dotyczy tylko mangio-crepe): Długość skoku odnosi się do czasu, jaki mówca potrzebuje, aby skoczyć do dramatycznego tonu. Krótsze skoki zajmują więcej czasu, ale są dokładniejsze pod względem tonu.",
    min=1,
    max=512,
    step=1,
    value=64
)
crepe_hop_widget.layout.visibility = "hidden"

save_interval_widget = widgets.IntSlider(
    min=1,
    max=50,
    step=1,
    description="Częstotliwość zapisu (save_every_epoch)",
    value=5,
    continuous_update=True,
    visible=True
)

epochs_widget = widgets.IntSlider(
    min=1,
    max=10000,
    step=1,
    description="Liczba epok treningu (total_epoch)",
    value=20,
    continuous_update=True
)

batch_size_widget = widgets.IntSlider(
    min=1,
    max=40,
    step=1,
    description="Rozmiar partii na jedną kartę (batch_size)",
    value=default_batch_size,
    continuous_update=True
)

save_latest_widget = widgets.Checkbox(
    description="Czy zapisać tylko najnowszą wersję pliku .ckpt, aby zaoszczędzić miejsce na dysku",
    value=True
)

cache_dataset_widget = widgets.Checkbox(
    description="Zapisz wszystkie zestawy treningowe w pamięci GPU. Buforowanie małych zestawów danych (mniej niż 10 minut) może przyspieszyć trening, ale buforowanie dużych zestawów danych zużyje dużo pamięci GPU i może nie zapewnić dużych korzyści ze względu na szybkość",
    value=False
)

save_small_model_widget = widgets.Checkbox(
    description="Zapisz mały model końcowy do folderu 'weights' przy każdym punkcie zapisu",
    value=True
)

pretrained_g_model = widgets.Textarea(
    rows=2,
    description="Ścieżka do wstępnie wytrenowanych wag modelu G",
    value="pretrained/f0G40k.pth"
)

pretrained_d_model = widgets.Textarea(
    rows=2,
    description="Ścieżka do wstępnie wytrenowanych wag modelu D",
    value="pretrained/f0D40k.pth"
)

gpu_info_widget = widgets.Text(
    description="Informacje o karcie graficznej",
    value=gpu_info
)

# pola na logi
output_info_widget1 = widgets.Textarea(
    description="Informacje wyjściowe",
    value=""
)

output_info_widget2 = widgets.Textarea(
    description="Informacje wyjściowe",
    value=""
)

output_info_widget3 = widgets.Textarea(
    description="Informacje wyjściowe",
    value=""
)

# progress bar
training_progress_bar = widgets.FloatProgress(
    value=0.0, 
    min=0.0, 
    max=1.0, 
    description='Postęp:', 
    bar_style='info', 
    orientation='horizontal'
)
training_progress_bar.layout.visibility = "hidden"

# nadaj klasy css
trainset_dir_widget.add_class("small-textfield")
exp_dir_widget.add_class("small-textfield")
gpu_num_widget.add_class("small-textfield")
pretrained_g_model.add_class("large-textfield")
pretrained_d_model.add_class("large-textfield")
output_info_widget1.add_class("large-textfield")
output_info_widget2.add_class("large-textfield")
output_info_widget3.add_class("large-textfield")
gpu_info_widget.add_class("large-textfield")
n_p_widget.add_class("input-slider")
spk_id_widget.add_class("input-slider")
crepe_hop_widget.add_class("input-slider")
save_interval_widget.add_class("input-slider")
epochs_widget.add_class("input-slider")
batch_size_widget.add_class("input-slider")
preprocess_btn.add_class("btn_container")
f0_process_btn.add_class("btn_container")
train_model_btn.add_class("btn_container")
train_index_btn.add_class("btn_container")
sr_widget.add_class("first-radio-hack")

css_stylesheet = """
<style>
.large-textfield, .small-textfield, .input-slider, .widget-radio {
	display: flex;
    flex-direction: column;
}
.large-textfield > textarea, .large-textfield > input {
    height: 90px !important;
    border-radius: 8px;
    border: silver solid 2px;
    box-shadow: silver 1px 1px 2px;
	order: 1;
    width: 100%;
}
.small-textfield > input {
    height: 40px !important;
    max-height: 40px !important;
	border-radius: 8px;
    border: silver solid 2px;
    box-shadow: silver 1px 1px 2px;
	order: 1;
	width: 100%;
}
.widget-label {
	width: auto !important;
	padding-bottom: 15px;
	text-align: left !important;
	white-space: normal !important;
	height: auto !important;
}
.input-slider > .widget-readout {
	order: 2;
}
.input-slider > .slider-container {
  order: 3;
  max-width: 80%;
  width: 250px;
  height: 30px !important;
  max-height: 30px !important;
}
.widget-radio-box {
    flex-direction: row !important;
	flex-wrap: wrap;
}
.widget-radio-box label {
    margin:5px !important;
    width: 120px !important;
}
.widget-radio > label {
	order: 1;
}
.widget-radio > .widget-radio-box {
	order: 2;
}
.widget-inline-hbox {
    height: auto !important;
    padding: 0px 15px 0px 15px;
}
.widget-checkbox > .widget-label {
    display: none;
}
.widget-checkbox > .widget-label-basic {
    white-space: normal !important;
}
.btn_container {
	border-radius: 8px;
    height: 40px
}
.widget-radio {
  max-height: 30px;
}
.first-radio-hack {
  max-height: 250px;
}
/* takie tam obejście, żeby odwróciło kolor tekstu na karcie w zależności od tła
.p-TabBar-tabLabel {
  background: inherit;
  background-clip: text;
  -webkit-background-clip: text;
  color: transparent;
  filter: invert(1) grayscale(1);
  -webkit-filter: invert(1) grayscale(1);
}
</style>
"""

tab = widgets.Tab()
tab.children = [
    widgets.VBox(
        children=[
            widgets.HTML(
                value = css_stylesheet
            ),
            widgets.VBox(
                children=[
                    widgets.HTML(
                        value="Krok 1: Wypełnij konfigurację treningu. Dane o treningu znajdują się w folderze logs, każdy model w osobnym folderze. Ręcznie wprowadź nazwę folderu, który zawiera konfigurację modelu, logi, pliki modeli uzyskane podczas treningu."
                    ),
                    widgets.HBox(
                        children=[
                            exp_dir_widget,
                            sr_widget,
                            rvc_ver_widget,
                        ]
                    ),
                    widgets.HBox(
                        children=[
                            pitch_guidance_widget,
                            n_p_widget
                        ]
                    ),
                ]
            ),
            widgets.VBox(
                children=[
                    widgets.HTML(
                        value="Krok 2a: Automatycznie przeglądaj wszystkie pliki w folderze z datasetem, które można zdekodować na pliki audio i przeprowadź normalizację fragmentów. Generuje 2 foldery wav w katalogu eksperymentu; tymczasowo obsługuje tylko trening jednej osoby."
                    ),
                    widgets.HBox(
                        children=[
                            widgets.VBox(
                                children=[
                                    trainset_dir_widget,
                                    spk_id_widget
                                ]
                            ),
                            widgets.VBox(
                                children=[
                                    preprocess_btn,
                                    output_info_widget1
                                ]
                            )
                        ]
                    )
                ]
            ),
            widgets.VBox(
                children = [
                    widgets.HTML(
                        value="Krok 2b: Wykorzystanie CPU do ekstrakcji wysokości dźwięku (jeśli model zawiera wsparcie dla wysokości dźwięku), wykorzystanie GPU do ekstrakcji cech (wybór numeru karty)"
                    ),
                    widgets.HBox(
                        children=[
                            widgets.VBox(
                                children=[
                                    gpu_num_widget,
                                    gpu_info_widget
                                ]
                            ),
                            widgets.VBox(
                                children=[
                                    f0_method_widget,
                                    crepe_hop_widget
                                ]
                            ),
                            widgets.VBox(
                                children=[
                                    f0_process_btn,
                                    output_info_widget2
                                ]
                            )
                        ]
                    )
                ]
            ),
            widgets.VBox(
                children = [
                    widgets.HTML(value="Krok 3: Wprowadź ustawienia treningu, rozpocznij trening modelu i indeksu"),
                    widgets.HBox(
                        children=[
                            widgets.VBox(
                                children=[
                                    save_interval_widget,
                                    epochs_widget,
                                    batch_size_widget
                                ]
                            ),
                            widgets.HBox(
                                children=[
                                    save_latest_widget,
                                    cache_dataset_widget,
                                    save_small_model_widget
                                ]
                            )
                        ]
                    ),
                    widgets.HBox(
                        children=[
                            widgets.HBox(
                                children = [
                                    widgets.VBox(
                                        children=[
                                            pretrained_g_model,
                                            pretrained_d_model
                                        ]
                                    ),
                                    widgets.VBox(
                                        children=[
                                            train_model_btn,
                                            train_index_btn
                                        ]
                                    ),
                                    output_info_widget3,
                                    training_progress_bar
                                ]
                            )
                        ]
                    )
                ]
            )
        ]
    )
]
tab.set_title(0, "Trening")
display(tab)

In [15]:
import ipywidgets as widgets
from IPython.display import display

def on_button_click(b):
    b.description = "Kliknąłeś mnie :D"
    print("Przycisk został kliknięty!")

button = widgets.Button(description="Kliknij mnie")
button.on_click(on_button_click)

display(button)


Button(description='Kliknij mnie', style=ButtonStyle())